# Datensammlung von Nachrichtenseiten per Web Scraping - Zeit Online

In [22]:
#Installation Bibliotheken

import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import json

In [23]:
#Definition der Parameter für das Web Scraping 

#Suchwörter in Google Search API
search_queries = ["Künstliche Intelligenz", "AI", "Artificial Intelligence", "KI"]

#Lege ein Startdatum für die automatische Datenerfassung fest
last_day_str = "2023-12-28"

#Zeitintervall von x Tagen vor dem Zieldatum
request_days = 3

#Limit für API-Anfragen
api_request_limit = 7

#Pausenzeit des Data Scraping über Beautiful Soup in Sekunden 
scrap_pause = 3

#Bereinigung der Datumsangabe
last_day = datetime.strptime(last_day_str, "%Y-%m-%d")

#Berechnung des letzten Tages
first_day = last_day - timedelta(days=request_days*api_request_limit) - ((timedelta(days=1)*api_request_limit)-timedelta(days=1))

#Ausgabe des Zeitraums
print(f"Time period: {first_day.strftime('%Y-%m-%d')} until {last_day_str} with {api_request_limit} requests")

Time period: 2023-12-01 until 2023-12-28 with 7 requests


In [24]:
#API-Anfrage

#API Schlüssel
api_key = "???"

#DataFrame für die Ergebnisse erstellen
df_data_all_queries = []

#Berechnung Zeitintervall für jede Schleife
loop_interval = timedelta(days=request_days)

#Schleife bis zur Erreichung der Grenze an API-Anfragen
for i in range(api_request_limit):
    #Berechnen des Start- und Enddatums für die aktuelle Schleife
    end_date = last_day - (loop_interval * i) - (timedelta(days=1) * i) if i > 0 else last_day - (loop_interval * i)
    start_date = end_date - timedelta(days=request_days)

    #Bereinigung von Datumsangaben
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    #Schleife über die Suchbegriffe
    for search_query in search_queries:
        #API-Endpunkt und Parameter
        api_endpoint = "https://www.googleapis.com/customsearch/v1"
        api_params = {
            "key": api_key,
            "cx": "???",  
            "q": f"{search_query} before:{end_date_str}",
            "sort": "date",
        }

        #API-Aufruf
        response = requests.get(api_endpoint, params=api_params)
        data = response.json()

        #Daten im DataFrame speichern
        if "items" in data:
            for item in data["items"]:
                link = item["link"]
                title = item["title"]

                #Extrahieren von HTML-Inhalten aus der Website
                article_response = requests.get(link)

                #Pause von x Sekunden
                time.sleep(scrap_pause)

                soup = BeautifulSoup(article_response.text, 'html.parser')

                '''Individueller Teil je Quelle'''

                #Extrahieren des Inhalts
                #Text aus 'p'-Elementen mit der Klasse 'paragraph article__item' extrahieren
                article_text = [p.get_text() for p in soup.find_all('p', class_='paragraph article__item')]

                #Verketten der extrahierten Textsegmente zu einem zusammenhängenden Artikeltext
                content = '\n'.join(article_text)

                #Extrahieren des Datums aus dem HTML-Inhalt
                meta_tag = soup.find('meta', {'name': 'date'})
                article_date = meta_tag.get('content', '') if meta_tag else ''

                #Prüfen, ob ein Wert für article_date vorhanden ist
                if article_date:
                    #Konvertieren des Datums in das gewünschte Format
                    formatted_date = datetime.strptime(article_date, "%Y-%m-%dT%H:%M:%S%z").strftime("%Y-%m-%d")
                else:
                    #Wenn article_date nicht vorhanden ist, wird formatted_date auf einen leeren String gesetzt.
                    formatted_date = "1900-01-01"

                #Extrahieren des Titels aus der HTML-Datei
                article_title = soup.title.text if soup.title else ""

                '''Individueller Teil je Quelle'''

                #Prüfen, ob das Datum mit dem Enddatum übereinstimmt
                if datetime.strptime(start_date_str, "%Y-%m-%d") <= datetime.strptime(formatted_date, "%Y-%m-%d") <= datetime.strptime(end_date_str, "%Y-%m-%d"):
                    df_data_all_queries.append({
                        "Suchbegriff": search_query,
                        "Quelle": "Zeit",
                        "Datum": formatted_date,
                        "Link": link,
                        "Titel": article_title,
                        "Text": content
                    })

#DataFrame für alle Suchbegriffe erstellen
df_all_queries = pd.DataFrame(df_data_all_queries)

#DataFrame anzeigen und prüfen, ob er leer ist
if df_all_queries.empty:
    print(f"Keine Einträge gefunden!")
else:
    print(df_all_queries)

                Suchbegriff Quelle       Datum  \
0    Künstliche Intelligenz   Zeit  2023-12-27   
1    Künstliche Intelligenz   Zeit  2023-12-27   
2    Künstliche Intelligenz   Zeit  2023-12-27   
3                        AI   Zeit  2023-12-27   
4                        AI   Zeit  2023-12-27   
..                      ...    ...         ...   
74  Artificial Intelligence   Zeit  2023-12-03   
75                       KI   Zeit  2023-12-04   
76                       KI   Zeit  2023-12-03   
77                       KI   Zeit  2023-12-02   
78                       KI   Zeit  2023-12-02   

                                                 Link  \
0   https://www.zeit.de/digital/2023-12/newyorktim...   
1   https://www.zeit.de/kultur/2023-11/kuenstliche...   
2   https://www.zeit.de/news/2023-12/27/new-york-t...   
3   https://www.zeit.de/digital/2023-12/newyorktim...   
4   https://www.zeit.de/kultur/2023-11/kuenstliche...   
..                                                ...   


In [25]:
#Neue Daten in einem Hauptdatensatz in CSV speichern

#Prüfen, ob die CSV-Datei bereits existiert
try:
    #Versuche, die vorhandene CSV-Datei zu lesen
    existing_df = pd.read_csv("2_Daten_Zeit_V2.csv")
except FileNotFoundError:
    #Wenn die Datei nicht existiert, erstelle einen leeren DataFrame
    existing_df = pd.DataFrame()

#Anhängen der neuen Daten an den bestehenden DataFrame
df = pd.DataFrame(df_all_queries)
df_to_append = pd.concat([existing_df, df], ignore_index=True)

#Duplikate auf Basis aller Spalten entfernen
df_to_append = df_to_append.drop_duplicates()

#Speichern des kombinierten DataFrame in der CSV-Datei
df_to_append.to_csv("2_Daten_Zeit_V2.csv", index=False)